# Семинар №6 - Вопросы сегментации изображений

In [ ]:
import cv2
import numpy as np
import sklearn
import skimage.segmentation
from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering
import skimage.io

import matplotlib.pyplot as plt

import time as time
import scipy as sp

In [ ]:
# вспомогательная функция
def plot_one_image(src_image, is_gray=False):
    """
    Отрисовать с помощью plt исходное изображение.
    
    :param src_image: np.ndarray: исходное изображение
    :param is_gray: bool: флаг для отображения ЧБ изображений
    :return: None
    """
    fig, m_axs = plt.subplots(1, 1, figsize=(6.4*2, 4.8*2), constrained_layout=False)
    ax1 = m_axs

    cmap = 'gray' if is_gray else None
    ax1.set_title('Исходное изображение')
    ax1.imshow(src_image, cmap=cmap)
    ax1.set_xticks([]), ax1.set_yticks([])
    plt.show()

In [ ]:
# вспомогательная функция
def plot_transform_result(src_image, transform_image, is_gray=False):
    """
    Отрисовать с помощью plt исходное изображение и его преобразование.
    
    :param src_image: np.ndarray: исходное изображение
    :param transform_image: np.ndarray: преобразованное изображение
    :param is_gray: bool: флаг для отображения ЧБ изображений
    :return: None
    """
    fig, m_axs = plt.subplots(1, 2, figsize=(6.4*2, 4.8*2), constrained_layout=False)
    ax1, ax2 = m_axs

    cmap = 'gray' if is_gray else None
    ax1.set_title('Исходное изображение')
    ax1.imshow(src_image, cmap=cmap)
    ax1.set_xticks([]), ax1.set_yticks([])
    ax2.set_title('Результат преобразования')
    ax2.imshow(transform_image, cmap=cmap)
    ax2.set_xticks([]), ax2.set_yticks([])
    plt.show()

# Вспомним про kNN 

In [ ]:
# Generate data
image = skimage.io.imread('img/cat.jpg')
image = skimage.transform.resize(image, tuple(map(int, np.array(image.shape[:2])/2)))
X = np.reshape(image, (-1, 1))

In [ ]:
X.shape

In [ ]:
plot_one_image(image)

In [ ]:
n_clusters = 3  # number of regions
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init=40, max_iter=500).fit(X)
label = np.reshape(kmeans.labels_, image.shape)

In [ ]:
# Plot the results on an image
cmap = plt.cm.get_cmap('Spectral', n_clusters)
rgba = cmap(np.linspace(0, 1, n_clusters))

plt.figure(figsize=(6.4*2, 4.8*2), constrained_layout=False)
plt.imshow(image, cmap=plt.cm.gray)
for l in range(n_clusters):
    tmp = label == l
    plt.contour(tmp[:, :, 0], contours=1,
               colors=[rgba[l]])
plt.xticks(())
plt.yticks(())
plt.show()

## Разница в представлении кодировки цвета

In [ ]:
image_ = skimage.color.rgb2hsv(image)
X = np.reshape(image_[:, :, 0], (-1, 1))

In [ ]:
n_clusters = 3  # number of regions
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, init='k-means++', 
                                n_init=40, algorithm='elkan',
                                max_iter=1000, n_jobs=-1).fit(X)
label = np.reshape(kmeans.labels_, image[:, :, 0].shape)

In [ ]:
# Plot the results on an image
cmap = plt.cm.get_cmap('Spectral', n_clusters)
rgba = cmap(np.linspace(0, 1, n_clusters))

plt.figure(figsize=(6.4*2, 4.8*2), constrained_layout=False)
plt.imshow(image, cmap=plt.cm.gray)
for l in range(n_clusters):
    tmp = label == l
    plt.contour(tmp[:, :], contours=1,
               colors=[rgba[l]])
plt.xticks(())
plt.yticks(())
plt.show()

# Agglomerative clustering

In [ ]:
# Generate data
image = skimage.io.imread('img/cat.jpg')
image = skimage.transform.resize(image, tuple(map(int, np.array(image.shape[:2])/2)))
X = np.reshape(image, (-1, 1))

## Algo

In [ ]:
# Define the structure A of the data. Pixels connected to their neighbors.
connectivity = grid_to_graph(*image.shape)

# Compute clustering
print("Compute structured hierarchical clustering...")
st = time.time()
ward = AgglomerativeClustering(n_clusters=3,
        linkage='ward', connectivity=connectivity, distance_threshold=None).fit(X)

label = np.reshape(ward.labels_, image.shape)
print("Elapsed time: ", time.time() - st)
print("Number of pixels: ", label.size)
print("Number of clusters: ", np.unique(label).size)

## Plot clustering result

In [ ]:
# Plot the results on an image
cmap = plt.cm.get_cmap('Spectral', np.unique(label).size)
rgba = cmap(np.linspace(0, 1, np.unique(label).size))

plt.figure(figsize=(6.4*2, 4.8*2), constrained_layout=False)
plt.imshow(image, cmap=plt.cm.gray)
for l in range(np.unique(label).size):
    tmp = label == l
    plt.contour(tmp[:, :, 0], contours=1,
               colors=[rgba[l]])
plt.xticks(())
plt.yticks(())
plt.show()

# Mean-shift clustering

In [ ]:
# Generate data
image = skimage.io.imread('img/cat.jpg')
image = skimage.transform.resize(image, tuple(map(int, np.array(image.shape[:2])/2)))

## Algo

In [ ]:
label = skimage.segmentation.quickshift(image, kernel_size=20, max_dist=6,
                                                 ratio=0.5,
                                                 convert2lab=False)

In [ ]:
n_clusters = np.unique(label).max()
print(n_clusters)

## Plot clustering result

In [ ]:
# Plot the results on an image
cmap = plt.cm.get_cmap('Spectral', n_clusters)
rgba = cmap(np.linspace(0, 1, n_clusters))

plt.figure(figsize=(6.4*2, 4.8*2), constrained_layout=False)
plt.imshow(image, cmap=plt.cm.gray)
for l in range(n_clusters):
    tmp = label == l
    plt.contour(tmp[:, :], contours=1,
               colors=[rgba[l]])
plt.xticks(())
plt.yticks(())
plt.show()

## Felzenszwalb algo

In [ ]:
label = skimage.segmentation.felzenszwalb(image, scale=100, sigma=0.5, min_size=50)

In [ ]:
n_clusters = np.unique(label).max()

# Plot the results on an image
cmap = plt.cm.get_cmap('Spectral', n_clusters)
rgba = cmap(np.linspace(0, 1, n_clusters))

plt.figure(figsize=(6.4*2, 4.8*2), constrained_layout=False)
plt.imshow(image, cmap=plt.cm.gray)
for l in range(n_clusters):
    tmp = label == l
    plt.contour(tmp[:, :], contours=1,
               colors=[rgba[l]])
plt.xticks(())
plt.yticks(())
plt.show()

# Метрики для сегментации задачи

Есть 2 способа кодировать предсказания в масках.

1. Предсказание для каждого класса записывать в отдельный канал предсказания. Если классов С, а размер исходного изображения H, W, то предсказание будет размера (H, W, C), где каждый канал (бинарный) означает результат классификации для каждого класса С.

2. Предсказание для каждого класса записывается в виде целого числа в один канал. Если классов С, а размер исходного изображения H, W, то предсказание будет размера (H, W), где значения предсказаний лежат в диапазоне [0, C].

Дальше мы будем придерживаться первого способа кодирования результата сегментации.

In [ ]:
# смоделируем пример маски сегментации после алгоритма

true_mask = np.zeros((400, 400, 3))  # истинная маска
true_mask[50:100, 30:370] = 1

pred_mask = np.zeros((400, 400, 3))  # предсказанная маска 
pred_mask[20:60, :] = 1

In [ ]:
# отобразим
plot_transform_result(true_mask, pred_mask)

## Dice метрика

In [ ]:
def dice_channel(probability, truth, threshold):
    # 
    batch_size = truth.shape[0]
    channel_num = truth.shape[3]
    mean_dice_channel = 0.
    all_channels = []
    for i in range(batch_size):
        for j in range(channel_num):
            channel_dice = dice_single_channel(probability[i, :, :, j], truth[i, :, :, j], threshold)
            all_channels.append(channel_dice)
            mean_dice_channel += channel_dice/(batch_size * channel_num)
    return mean_dice_channel, all_channels


def dice_single_channel(probability, truth, threshold, eps=1E-9):
    # 
    t = (truth.flatten() > 0.5)
    p = (probability.flatten() > threshold)
    dice = (2.0 * (p * t).sum() + eps) / (p.sum() + t.sum() + eps)
    return dice

In [ ]:
mean_dice, all_dice = dice_channel(np.array([pred_mask]), np.array([true_mask]), 0.5)

print('Mean Dice:', mean_dice)
print('Dice per channel:', all_dice)

## Jaccard score

In [ ]:
def jaccard_score(preds, labels, C, EMPTY=1., ignore=None, per_image=False):
    """
    Array of IoU for each (non ignored) class
    """
    if not per_image:
        preds, labels = (preds,), (labels,)
    ious = []
    for pred, label in zip(preds, labels):
        iou = []
        for i in range(C):
            if i != ignore: # The ignored label is sometimes among predicted classes (ENet - CityScapes)
                intersection = ((label == i) & (pred == i)).sum()
                union = ((label == i) | ((pred == i) & (label != ignore))).sum()
                if not union:
                    iou.append(EMPTY)
                else:
                    iou.append(float(intersection) / union)
        ious.append(iou)
    ious = map(np.mean, zip(*ious)) # mean accross images if per_image
    return np.array(tuple(ious))

In [ ]:
iou = jaccard_score(np.array([pred_mask]), np.array([true_mask]), C=2, EMPTY=0, per_image=False)

print('Mean IoU:', iou)
# print('Dice per channel:', all_dice)

# Naive Bayes Classifier

Воспользуемся моделью наивного байесовского классификатора для сегментации - воспользуемся постановкой задачи, что требуется классифицировать каждый пиксель.

In [ ]:
path = "img/test2-mini.jpg"
arr = skimage.io.imread(path)

plot_one_image(arr)

## Формируем обучающую выборку

In [ ]:
# априорная информация
paths = ["img/Sky.jpg", "img/Road.jpg", "img/Grass.jpg"]
initial_probability = {"img/Sky.jpg": 0.30, "img/Road.jpg": 0.20, "img/Grass.jpg": 0.50}

# сохраняем результаты расчета априорной информации
number_of_pixels = arr.size
class_info = []

for path in paths:
    tmp_arr = skimage.io.imread(path)
 
    # расчет распределения признака № 1 (значения пикселя - RGB)
    class_mean = np.array([np.mean(tmp_arr[..., i]) for i in range(3)])
    class_var = np.array([np.var(tmp_arr[..., i]) for i in range(3)])
    
    # априорная вероятность класса
    class_freq = len(tmp_arr)
    class_probabilty = class_freq / number_of_pixels 
    
    class_info.append([initial_probability[path], class_mean, class_var])

print("class_info")
print(class_info)

## Строим модель

In [ ]:
# многомерное распеределение Гаусса
from scipy.stats import multivariate_normal

In [ ]:
def naive_bayes_predict (arr, class_info, fixed_pixels_index=[], correct_arr = []):
    predict_array = np.zeros((len(arr), len(arr[0])), dtype=float)
    
    class_color = [0, 127, 255]  # цвет класса для отображения
    
    # попиксельная классификация
    for i in range(0, len(arr)):
        for j in range(0, len(arr[0])): 
            if (len(fixed_pixels_index) > 0 and len(correct_arr) > 0 and fixed_pixels_index[i][j] == 1):
                predict_array[i][j] = correct_arr[i][j]
                continue
                
            max_probabilty = 0
            best_class = -1
            val = arr[i][j]  # текущее значение пикселя
            
            # проверить вероятность для каждого класса и выбрать наиболее вероятнный
            for cls_index in range(len(class_info)):
                cls_p = class_info[cls_index][0]  # априорная вероятность
                mean = class_info[cls_index][1]  # среднее признака 
                var = class_info[cls_index][2]  # вариация признака


                pos = multivariate_normal.pdf(val, mean, var)  # вероятность 
                cls_posterior = cls_p * pos

                if (cls_posterior > max_probabilty):
                    max_probabilty = cls_posterior
                    best_class = cls_index
            
            predict_array[i][j] = class_color[best_class]
            
    return predict_array

In [ ]:
# расчет
initial_arr = naive_bayes_predict(arr, class_info)

In [ ]:
plot_transform_result(arr, initial_arr)